Instalando dependencias

In [ ]:
%pip install autogluon
%pip install --upgrade ipykernel
%pip install pandas
%pip install scikit-learn
%pip install tqdm
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install ipywidgets
%pip install bokeh

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.9 kB ? eta -:--:--
   ------------ --------------------------- 92.2/294.9 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 286.7/294.9 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Importando dependencias

In [9]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

from sklearn.metrics import f1_score, roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, KFold
from sklearn.calibration import CalibratedClassifierCV

from autogluon.tabular import TabularPredictor

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

Load CSVs

In [14]:
train = pd.read_csv('input/train.csv', index_col='id').reset_index(drop=True)
test = pd.read_csv('input/test.csv', index_col='id').reset_index(drop=True)
sample_submission = pd.read_csv('input/submission_sample.csv')
meta = pd.read_csv('input/metadata.csv')

cat_nom = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Qualitativo nominal")].iloc[:,0]]
cat_ord = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Qualitativo ordinal")].iloc[:,0]]
num_dis = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Quantitativo discreto")].iloc[:,0]]
num_con = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Quantitativo continua")].iloc[:,0]]

In [18]:
print(num_con)

['var55', 'var56', 'var57', 'var58', 'var59', 'var60', 'var61', 'var62', 'var63', 'var64', 'var65', 'var66']


Iniciar preparação

In [19]:
K=10
SEED=25
kf = KFold(n_splits=K, random_state=SEED, shuffle=True)

In [20]:
print(kf)

KFold(n_splits=10, random_state=25, shuffle=True)


Get threshold

In [21]:
def get_threshold(y_true, y_pred):
    # Moving threshold
    thresholds = np.arange(0.0, 1.0, 0.01)
    f1_scores = []
    for thresh in thresholds:
        f1_scores.append(
            f1_score(y_true, [1 if m>thresh else 0 for m in y_pred]))
    f1s = np.array(f1_scores)
    return thresholds[f1s.argmax()]


def custom_f1(y_true, y_pred):

    max_f1_threshold =  get_threshold(y_true, y_pred)

    y_pred = np.where(y_pred>max_f1_threshold, 1, 0)
    f1_after = f1_score(y_true, y_pred)

    return f1_after

init predictor

In [45]:
predictor = TabularPredictor(label="y",
                             problem_type='binary',
                             eval_metric="log_loss",
                             path='./AutoGlon/',
                             verbosity=1)
hyperparameters = predictor.get_hyperparameter_config('default')
#predictor.fit(train, presets='medium_quality')

#results = predictor.fit_summary()

AttributeError: 'TabularPredictor' object has no attribute 'get_hyperparameter_config'

2024-04-23 22:08:16,960	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::_sub_fit() (pid=14248, ip=127.0.0.1)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aluno\Desktop\files_ml\.conda\Lib\site-packages\autogluon\common\loaders\load_pkl.py", line 43, in load
    with compression_fn_map[compression_fn]["open"](validated_path, "rb", **compression_fn_kwargs) as fin:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './AutoGlon/ds_sub_fit\\sub_fit_ho\\models\\KNeighborsUnif_BAG_L1\\utils\\oof.pkl'

During handling of the above exception, another exception occurred:

ray::_sub_fit() (pid=14248, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1883, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1984, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx

Get predictions

In [37]:
predictor.get_model_names()

C:\Users\aluno\AppData\Local\Temp\ipykernel_11476\997680543.py:1: DeprecationWarning: `get_model_names` has been deprecated and will be removed in version 1.2. Please use `model_names` instead. This will raise an error in the future!
  predictor.get_model_names()


['KNeighborsUnif',
 'KNeighborsDist',
 'LightGBMXT',
 'LightGBM',
 'RandomForestGini',
 'RandomForestEntr',
 'CatBoost',
 'ExtraTreesGini',
 'ExtraTreesEntr',
 'NeuralNetFastAI',
 'XGBoost',
 'NeuralNetTorch',
 'LightGBMLarge',
 'WeightedEnsemble_L2']

In [39]:
y_oof = predictor.predict_proba_oof(model='KNeighborsUnif').iloc[:,1]
y_pred = predictor.predict_proba(test).iloc[:,1]

AssertionError: Predictor must be in bagged mode to get out-of-fold predictions.

check threshold between predict value and train value

In [36]:
final_threshold = get_threshold(train.y, y_oof)

ValueError: Found input variables with inconsistent numbers of samples: [14123, 21183]

In [ ]:
print(final_threshold)